In [1]:
# 下载数据集并预处理

from datasets import load_dataset
import datasets

dataset = load_dataset("yelp_review_full")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 采用全量数据
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [2]:
# 微调训练配置

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3)

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# 开始训练

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\mtide\miniconda3\envs\py311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.756800,0.757306,0.671940
2,0.688800,0.728346,0.684720
3,0.597000,0.734912,0.689660


TrainOutput(global_step=121875, training_loss=0.7132316257261618, metrics={'train_runtime': 44625.6792, 'train_samples_per_second': 43.697, 'train_steps_per_second': 2.731, 'total_flos': 5.130803778048e+17, 'train_loss': 0.7132316257261618, 'epoch': 3.0})

In [4]:
# 验证模型指标

trainer.evaluate(eval_dataset)

{'eval_loss': 0.7349115014076233,
 'eval_accuracy': 0.68966,
 'eval_runtime': 424.246,
 'eval_samples_per_second': 117.856,
 'eval_steps_per_second': 14.732,
 'epoch': 3.0}

In [5]:
# 保存模型和训练状态

trainer.save_model(model_dir)
trainer.save_state()